In [98]:
import numpy as np
import pandas as pd
import sklearn.linear_model
import sklearn.metrics
import os

## Load Data

In [99]:
data_dir = 'data_reviews'
x_train_df = pd.read_csv(os.path.join(data_dir, 'x_train.csv'))
y_train_df = pd.read_csv(os.path.join(data_dir, 'y_train.csv'))

N, n_cols = x_train_df.shape
print("Shape of x_train_df: (%d, %d)" % (N,n_cols))
print("Shape of y_train_df: %s" % str(y_train_df.shape))
x_train_df

Shape of x_train_df: (2400, 2)
Shape of y_train_df: (2400, 1)


,website_name,text
0,amazon,Oh and I forgot to also mention the weird colo...
1,amazon,THAT one didn't work either.
2,amazon,Waste of 13 bucks.
3,amazon,"Product is useless, since it does not have eno..."
4,amazon,None of the three sizes they sent with the hea...
...,...,...
2395,yelp,The sweet potato fries were very good and seas...
2396,yelp,I could eat their bruschetta all day it is dev...
2397,yelp,Ambience is perfect.
2398,yelp,We ordered the duck rare and it was pink and t...


In [100]:
# df to list
tr_text_list = x_train_df['text'].values.tolist()

In [101]:
y_train_list = y_train_df['is_positive_sentiment'].values.tolist()
len(y_train_list)

2400

In [102]:
y_train_list

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


## Partition sentences into words

In [103]:
from sklearn.feature_extraction.text import CountVectorizer

In [104]:
vectorizer = CountVectorizer()

### How big is my vocabulary?

In [105]:
x = vectorizer.fit_transform(tr_text_list)
vocab = vectorizer.get_feature_names_out()
vocab.shape[0]

4510

In [106]:
x_train_NV = x.toarray()
x_train_NV.shape # N is the number of sentences, and V is the number of vocabulary

(2400, 4510)

In [107]:
N = x_train_NV.shape[0]
V = x_train_NV.shape[1]
N,V

(2400, 4510)

In [108]:
vectorizer.vocabulary_.get('the')

3941

### Get the common and rare

In [109]:
x_train_NV[0][2701] # search the first sentence for 'oh'

1

In [110]:
x_common = []

x_col_sum = np.sum(x_train_NV, axis = 0)

np.max(x_col_sum)

1572

In [111]:
x_col_sum.shape, x_col_sum[2701]

((4510,), 9)

In [112]:
x_common = x_col_sum > N/2 # happens half of the time 
x_common_V = [int(x) for x in x_common]
x_common_V = np.array(x_common_V)

In [113]:
x_rare = x_col_sum < 10
x_rare_V = [int(x) for x in x_rare]
x_rare_V = np.array(x_rare_V)

In [114]:
x_common_V.shape, x_rare_V.shape

((4510,), (4510,))

In [115]:
# x_common_V.reshape(V, 1)

In [116]:
x_common = x_train_NV * x_common_V 
x_rare = x_train_NV * x_rare_V

idx = vectorizer.vocabulary_.get('good')
idx

1752

In [117]:
np.sum(x_train_NV[:, idx])

185

In [118]:
x_rare.shape, x_common.shape

((2400, 4510), (2400, 4510))

### Get Normal

In [119]:
x_normal = x_train_NV * (1 - x_common_V) * (1 - x_rare_V)

In [120]:
x_normal[:, idx]

array([0, 0, 0, ..., 0, 0, 1])

In [121]:
y_train_list = np.array(y_train_list)
y_train_list

array([0, 0, 0, ..., 1, 1, 1])

In [122]:
# x_normal_copy = np.copy(x_normal)
# x_normal_copy[0].shape, x_common_V.shape

In [123]:
# x = []

In [124]:
# y_train_list.shape, x_common_V.shape[0]

In [125]:
# for i in range(x_common_V.shape[0]):
#     if x_common_V[i] == 1:
#         x_normal_copy = np.delete(x_normal_copy, i, axis=1)
    
    

In [126]:
# x_normal_copy.shape

In [127]:
# t = {[0,0,0,0],[0,1,0,1]}

In [128]:
%run performance_metrics.py
%run cross_validation.py

In [129]:
# calc_root_mean_squared_error()
# train_models_and_calc_scores_for_n_fold_cv
# make_train_and_test_row_ids_for_n_fold_cv

In [130]:
M = 2

x_tr_M2 = np.random.randint(2, size=10).reshape(-1, 1)
x_va_N2 = np.random.randint(2, size=10).reshape(-1, 1)
y_tr_M = np.random.randint(2, size=10)
y_va_N = np.random.randint(2, size=10)
x_tr_M2, x_va_N2, y_tr_M, y_va_N

(array([[0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1]]),
 array([[0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0]]),
 array([1, 1, 0, 1, 0, 1, 1, 1, 0, 1]),
 array([0, 1, 1, 0, 0, 0, 0, 1, 0, 1]))

## Cross Validation

In [131]:
%run binary_metrics.py
%run proba_metrics.py

In [132]:
K = 3 # num folds of CV
random_state = 1234

train_error_per_fold = np.zeros(K, dtype=np.float32)
test_error_per_fold = np.zeros(K, dtype=np.float32)

N = x_normal.shape[0]
# TODO define the folds here by calling your function
# e.g. ... = make_train_and_test_row_ids_for_n_fold_cv(...)
train_ids, test_ids = make_train_and_test_row_ids_for_n_fold_cv(
    N, K, random_state)

len(train_ids[0]), len(test_ids[0])

(1599, 801)

In [133]:
y_list = y_train_list

In [134]:
y_list

array([0, 0, 0, ..., 1, 1, 1])

In [139]:
# partition the data into train and test

x_tr, y_tr = list(), list()
y_va, x_va = list(), list()


for i in range(len(train_ids)):
    x_tr.append(x_normal[train_ids[i]])
    y_tr.append(y_list[train_ids[i]])

for i in range(len(test_ids)):
    x_va.append(x_normal[test_ids[i]])
    y_va.append(y_list[test_ids[i]])


len(x_tr[0])

1599

In [136]:
C_grid = np.logspace(-9, 6, 31)

In [137]:
x_tr[0].shape

(1599, 4510)

In [138]:
K

3

In [140]:
cv_tr_bce_list = list()
cv_va_bce_list = list()
cv_te_bce_list = list()
    
# idx = 0
for C in C_grid:

    model_F3_list = list()

    # Allocate lists for storing BCE metrics
    tr_bce_list = list()
    va_bce_list = list()
    te_bce_list = list()

    # Allocate lists for storing ERROR RATE metrics
    tr_err_list = list()
    va_err_list = list()
    te_err_list = list()

    # idx = idx + 1
    for i in range(K):
        xtr = x_tr[i]
        ytr = y_tr[i]
        xva = x_va[i]
        yva = y_va[i]

        lr_F3 = sklearn.linear_model.LogisticRegression(C = C, penalty = 'l2', solver='lbfgs', max_iter=2000) # TODO fixme
        lr_F3.fit(xtr, ytr) # TODO fixme

        model_F3_list.append(lr_F3)

        yproba1_tr_M = lr_F3.predict_proba(xtr)[:,1] # The probability of predicting class 1 on the training set
        yproba1_va_N = lr_F3.predict_proba(xva)[:,1] # The probability of predicting class 1 on the validation set

        # Compute error rate aka zero-one loss
        my_tr_err = sklearn.metrics.zero_one_loss(ytr, yproba1_tr_M >= 0.5)
        my_va_err = sklearn.metrics.zero_one_loss(yva, yproba1_va_N >= 0.5)
        tr_err_list.append(my_tr_err)
        va_err_list.append(my_va_err)

        # TODO: using the calc_mean_binary_cross_entropy_from_probas() function from above:
        # Part c: calculate the binary cross entropy (bce) on the training set
        # Part d: calculate the binary cross entropy (bce) on the validation set
        my_tr_bce = calc_mean_binary_cross_entropy_from_probas(ytr, yproba1_tr_M) # TODO fixme
        my_va_bce = calc_mean_binary_cross_entropy_from_probas(yva, yproba1_va_N) # TODO fixme
        # Save bce for future selection on Models.
        tr_bce_list.append(my_tr_bce) 
        va_bce_list.append(my_va_bce) 
    
    
    # best_C_idx = np.argmin(va_bce_list)
    # best_C__lrF3 = C_grid[best_C_idx]
    
    cv_tr_bce_list.append((sum(tr_bce_list)/len(tr_bce_list)))
    cv_va_bce_list.append((sum(va_bce_list)/len(va_bce_list)))
    


In [143]:
cv_va_bce_list, cv_tr_bce_list

([1.0000308533456967,
  1.0001782150465923,
  1.0002538666942764,
  1.0002653418282195,
  1.0002644454793943,
  1.000261612468295,
  1.000252666319325,
  1.0002244222506789,
  1.0001348768152072,
  0.9998524606730181,
  0.9989671510935278,
  0.996238456677342,
  0.9882373229533575,
  0.9675027275009862,
  0.9247265106573018,
  0.8590923204292652,
  0.7839306395892978,
  0.722084365208652,
  0.7019856174990542,
  0.7466237336124953,
  0.8629140685623665,
  1.0409572086598144,
  1.254905335524872,
  1.4547513809215495,
  1.6054737456916508,
  1.7045187523376952,
  1.7774938718966657,
  1.8407937113877209,
  1.8880146420609967,
  1.9189757958199474,
  1.941470436789988],
 [0.9999860996547977,
  0.999961968144777,
  0.999948048908716,
  0.9999462840302135,
  0.9999452043439051,
  0.9999417901777495,
  0.9999309946393641,
  0.9998968651233836,
  0.9997890209155488,
  0.9994488155688099,
  0.9983813940872462,
  0.9950861284569558,
  0.9853795717888439,
  0.9599096270800254,
  0.9058586976916

In [96]:
best_model = model_F3_list[np.argmin(cv_tr_bce_list)]

In [ ]:
# remove punctuation in the text
def rm_punc(text_list):
    puncs = '!@#$%^&*()_+-={}[]:;|<>,.?/"\''
    clean_text_list = []
    for text in text_list:
        clean_text = ""
        for char in text:
            if char not in puncs:
                clean_text += char.lower()
        clean_text_list.append(clean_text)
    return clean_text_list

clean_tr_text = rm_punc(tr_text_list)
clean_tr_text[0]

In [ ]:
# partition the data
words_list = []
for text in clean_tr_text:
    words_list.append(text.split())
    
words_list[0]